# FairWorkflows execution demo

## Define the steps of your workflow
Each step should be its own function. Mark the function as such with the @fairstep decorator.

In [1]:
from fairworkflows import is_fairworkflow, is_fairstep

In [18]:
@is_fairstep(label='Addition')
def add(a:float, b:float) -> float:
    """Adding up numbers!"""
    return a + b

In [19]:
@is_fairstep(label='Subtraction')
def sub(a: float, b: float) -> float:
    """Subtracting numbers."""
    return a - b

In [20]:
@is_fairstep(label='Multiplication')
def mul(a: float, b: float) -> float:
    """Multiplying numbers."""
    return a * b

In [21]:
@is_fairstep(label='A strange step with little use')
def weird(a: float, b:float) -> float:
    """A weird function"""
    return a * 2 + b * 4
    

## Define your workflow using @fairworkflow
Now write a function which describes your workflow. Mark this function with the @fairworkflow decorator.

In [22]:
@is_fairworkflow(label='My Workflow')
def my_workflow(in1, in2, in3):
    """
    A simple addition, subtraction, multiplication workflow
    """
    t1 = add(in1, in2)
    t2 = sub(in1, in2)
    t3 = mul(weird(t1, in3), t2)
    return t3

## Create an instance of your workflow and display it

In [23]:
fw = my_workflow(1, 4, 3)
type(fw)

fairworkflows.fairworkflow.FairWorkflow

In [24]:
fw.display()

| workflow |
| --- |
| ![workflow workflow](control-workflow.svg) |

## Publish the (prospective) workflow
You may publish the workflow, and its steps, as nanopublications in the usual manner:

In [9]:
#fw.publish_as_nanopub()

Be warned though - the above will keep publishing to the 'real' nanopub server network. For testing you may prefer to publish to the test servers as follows (note that this will refuse to publish a workflow you have already published :

In [10]:
fw.publish_as_nanopub(use_test_server=True)

Published to http://purl.org/np/RAVNClKvLDlA07954NJz4E8XnfRf4KAgMu-ZsLIHz-Cno
Published concept to http://purl.org/np/RAVNClKvLDlA07954NJz4E8XnfRf4KAgMu-ZsLIHz-Cno#step
Published to http://purl.org/np/RA3NuSBHbN4625tyY-OWCLlS8hvsoy0GDavo1zGVLlNEA
Published concept to http://purl.org/np/RA3NuSBHbN4625tyY-OWCLlS8hvsoy0GDavo1zGVLlNEA#step
Published to http://purl.org/np/RAiw6NzI5aJNCXKTYHtxBU28qt5iO8rqZDQ8sxwvnLIdU
Published concept to http://purl.org/np/RAiw6NzI5aJNCXKTYHtxBU28qt5iO8rqZDQ8sxwvnLIdU#step
Published to http://purl.org/np/RAm0zCXUpBz-12csqLaCkRzjgXUDd9uXkzADG2Wi7HcGo
Published concept to http://purl.org/np/RAm0zCXUpBz-12csqLaCkRzjgXUDd9uXkzADG2Wi7HcGo#step
Published to http://purl.org/np/RARrWQq8rTQvXEFB4F2cmR8rf7r0oWct-T4CEle2BBAVw
Published concept to http://purl.org/np/RARrWQq8rTQvXEFB4F2cmR8rf7r0oWct-T4CEle2BBAVw#plan


{'nanopub_uri': 'http://purl.org/np/RARrWQq8rTQvXEFB4F2cmR8rf7r0oWct-T4CEle2BBAVw',
 'concept_uri': 'http://purl.org/np/RARrWQq8rTQvXEFB4F2cmR8rf7r0oWct-T4CEle2BBAVw#plan'}

You can then find your nanopublications by replacing the base of the URI with http://test-server.nanopubs.lod.labs.vu.nl/

## Execute your workflow using .execute()
Set num_threads greater than 1 if you wish to exploit parallelisation in your workflow. The retrospective provenance is also returned as a (nano) Publication object, that can optionally be published.

In [11]:
result, prov = fw.execute(num_threads=2)
result

-66

 ## Retrospective prov
 The retrospective prov object is not yet implemented, pending one of the RDF tasks set out in the planning document. For now, you may see simply the provenance trace for this run. We wish to turn this into a (Plex) RDF nanopublication:

In [12]:
type(prov)

nanopub.publication.Publication

In [13]:
print(prov)

Original source URI = None
@prefix : <http://purl.org/nanopub/temp/mynanopub#> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:provenance {
    :assertion prov:generatedAtTime "2021-01-29T21:38:54.494243"^^xsd:dateTime .
}

:pubInfo {
    : prov:generatedAtTime "2021-01-29T21:38:54.494243"^^xsd:dateTime ;
        prov:wasAttributedTo <https://orcid.org/0000-0000-0000-0000> .
}

:assertion {
    :retroprov a prov:Activity ;
        rdfs:label """2021-01-29 21:38:54,484 - job             1: add(1, 4)
2021-01-29 21:38:54,484 - job             2: sub(1, 4)
2021-01-29 21:38:54,487 - result          1 [add(1, 4)]: done -> 5
2021-01-29 21:38:54,487 - job             3: weird(5, 3)
2021-01-29 21:38:54,487 - result          2 [sub(1, 4)]: done -> -3
2021-01-29 21:38:54,488 - result          3 [weird(5, 3)]: done -> 22
2021-01-29 21:38:54,488

### Provide semantic annotations for input and output variables
If you wish to specify semantic types for the inputs/outputs to a step, you can do so in the arguments to the decorator.
For example, if you have an input parameter 'a', you can write a='http://www.example.org/distance' to assign that (semantic) type to a. As output of functions is not named in python, you can specify the same but with 'out1', 'out2' etc. See the following example:

In [41]:
@is_fairstep(label='Addition', a='http://www.example.org/distance', out1='http://www.example.org/mass')
def add(a:float, b:float) -> float:
    return a + b

If we now look at the RDF generated for the step, we will see that input parameter 'a' and the step output ('out1') both have the (additional) semantic types specified.

In [42]:
print(add._fairstep)

Step URI = http://www.example.org/unpublished-add
@prefix bpmn: <http://dkm.fbk.eu/index.php/BPMN2_Ontology#> .
@prefix ns1: <http://purl.org/dc/terms/> .
@prefix pplan: <http://purl.org/net/p-plan#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

_:N93427078e7bb4e038a28a81213850ff2 {
    [] a bpmn:ScriptTask,
            pplan:Step ;
        rdfs:label "Addition" ;
        ns1:description """@is_fairstep(label='Addition', a='http://www.example.org/distance', out1='http://www.example.org/mass')
def add(a:float, b:float) -> float:
    return a + b
""" ;
        pplan:hasInputVar [ a pplan:Variable,
                    <http://www.example.org/distance> ;
                rdfs:label "a" ;
                rdfs:comment "float" ],
            [ a pplan:Variable ;
                rdfs:label "b" ;
                rdfs:comment "float" ] ;
        pplan:hasOutputVar [ a pplan:Variable,
                    <http://www.example.org/mass> ;
                rdfs:label "out1" ;
             

### Specify more than one semantic type for a parameter
You can provide a list of URIs if you want to specify several semantic types for e.g. parameter 'a':

In [45]:
@is_fairstep(label='Addition', a=['http://www.example.org/distance', 'http://www.example.org/number'])
def another_step(a:float, b:float) -> float:
    return a + b

In [46]:
print(another_step._fairstep)

Step URI = http://www.example.org/unpublished-another_step
@prefix bpmn: <http://dkm.fbk.eu/index.php/BPMN2_Ontology#> .
@prefix ns1: <http://purl.org/dc/terms/> .
@prefix pplan: <http://purl.org/net/p-plan#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

_:N8cb43b17b8614a6aae5daa3ed5b8459f {
    [] a bpmn:ScriptTask,
            pplan:Step ;
        rdfs:label "Addition" ;
        ns1:description """@is_fairstep(label='Addition', a=['http://www.example.org/distance', 'http://www.example.org/number'])
def another_step(a:float, b:float) -> float:
    return a + b
""" ;
        pplan:hasInputVar [ a pplan:Variable,
                    <http://www.example.org/distance>,
                    <http://www.example.org/number> ;
                rdfs:label "a" ;
                rdfs:comment "float" ],
            [ a pplan:Variable ;
                rdfs:label "b" ;
                rdfs:comment "float" ] ;
        pplan:hasOutputVar [ a pplan:Variable ;
                rdfs:label "ou